## WHAT'S THE PROBLEM WE ARE TRYING TO SOLVE?

We wish to quantify the risk of a portfolio where the timeseries of relevant asset values exhibits multicollinearity (look it up, e.g. [on wikipedia](https://en.wikipedia.org/wiki/Multicollinearity)).

In this example our domain is US yield to maturities and out portfolio is a position in the 7 year asset. We'll assume that the cheapest hedging instruments are at the 2y, 5y, 10y and 30y points.

The following code uses coppertop-bones.

<br>

*Concepts to become familiar with...* \
Risk - what is it? \
Risk-factors \
Model, modelling, model risk, modelling risk, implementation risk. \
Supervised vs unsupervised learning. What are the implications of each? \
Why are residuals important? \
Over-fitting, data splitting (& carve-out?), cross-validation, statistical tests.

<br>

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
import datetime, numpy as np, plotnine, statsmodels.api as sm, scipy.stats, sys
from numpy import linalg

from coppertop.pipe import *

from dm.core.types import btup, bframe, date, num, count as tCount, pylist, T1, txt, offset, pytuple, bstruct, \
    index, void, matrix, N
from dm.ols import OLSResult
from dm.jupyter import P9
from dm.panda import panda

from bones.core.sentinels import Void
from bones.lang.structs import tvarray

import dm.linalg.core, dm.pp, dm.panda, dm.jupyter, dm.plot
from broot import *
from broot import dm, seaborn

In [3]:
from bones.lang.metatypes import BTNom
from bones.lang.structs import tv

p9 = BTNom.ensure('p9').setCoercer(tv)

@coppertop
def PP(p: p9) -> p9:
    fig = p._v.draw(show=True)
    return p

array_ = (N**num)&tvarray
DATA_PATH = "./data"

<br>

#### UTILS

In [4]:
@coppertop
def residualsPlot(res:array_) -> P9:
    # res = res >> to >> (vec&tvarray)
    df = panda({'i':range(res >> count), 'res':res})
    answer = (plotnine.ggplot(df, plotnine.aes(x='i', y='res')) + plotnine.geom_point())
    answer = answer | P9
        # + p9.geom_line(color='lightgrey', size=0.25)
    return answer


@coppertop
def showStats(Y, X):
    return showStats(Y, X, dict(addIntercept=False))

@coppertop
def showStats(Y, X, options):
    lm = dm.stats.ols(Y, X, options)
    f'beta:    {(lm >> betaHat >> round(_, 3)).reshape(lm.K)}' >> PP #      resDoF: {lm.resDoF}' >> PP
    f't-stats: {lm >> tStats >> round(_, 3)} {lm >> tVals >> round(_, 3)}' >> PP
    f'F-stat:  {np.round(lm >> fStat, 1)} (5% is > {np.round(lm >> fCrit(_,0.05), 1)}), {np.round(lm >> fVal, 4)}    R2: {np.round(lm >> r2, 3)}   predR2: {np.round(dm.stats.predictedR2(Y, X, options), 3)}' >> PP
    return lm

@coppertop
def showResStats(x):
    av = np.mean(res); st = np.std(res); sk = scipy.stats.skew(res); ku = scipy.stats.kurtosis(res)
    des = scipy.stats.describe(res)
    skt = scipy.stats.skewtest(res)
    kut = scipy.stats.kurtosistest(res)
    sk = des.skewness if skt.pvalue < 0.05 else 0.0
    ku = des.kurtosis if kut.pvalue < 0.05 else 0.0
    f"res - mean: {format(des.mean, '.4f')}  std: {format(np.sqrt(des.variance), '.4f')}  skew: {format(sk, '.4f')}  kurt: {format(ku, '.4f')}" >> PP


<br>

#### LOAD DATA

In [5]:
renames = {
    'Date':'date', '1 Mo':'_1m', '2 Mo':'_2m', '3 Mo':'_3m', '6 Mo':'_6m', '1 Yr':'_1y', '2 Yr':'_2y', 
    '3 Yr':'_3y', '5 Yr':'_5y', '7 Yr':'_7y', '10 Yr':'_10y', '20 Yr':'_20y', '30 Yr':'_30y',
}

conversions = dict(
    date=to(_, (N**date)&tvarray, 'MM/DD/YY'), _1m=to(_,array_), _2m=to(_,array_), _3m=to(_,array_),
    _6m=to(_,array_), _1y=to(_,array_), _2y=to(_,array_), _3y=to(_,array_), _5y=to(_,array_),
    _7y=to(_,array_), _10y=to(_,array_), _20y=to(_,array_), _30y=to(_,array_),
)

path = DATA_PATH
filename = 'us yields.csv'
ytms = (path + '/' + filename) >> dm.frame.fromCsv(_, renames, conversions)

select three months worth and calc diffs

In [6]:
d1 = '2021.01.01' >> to(_, _, 'YYYY.MM.DD') >> date
d2 = '2021.04.01' >> to(_, _, 'YYYY.MM.DD') >> date

subset = ytms >> select >> (lambda r: d1 <= r.date and r.date < d2)
usDiffs = subset >> keys >> drop >> 'date' \
    >> inject(_, bframe(), _) >> (lambda p, name:
        p >> atPut >> name >> (subset >> at >> name >> diff)
    )

# TODO something like the following is clearer? 
# usDiffs = subset >> drop >> 'date' >> bycols >> collect >> lambda(name, col: col >> vec_.diff)

<br>

### OLS

Let's try the following models

In [7]:
ytm2y = usDiffs >> take >> '_2y' >> takePanel
ytm5y = usDiffs >> take >> '_5y' >> takePanel
ytm7y = usDiffs >> take >> '_7y' >> takePanel
ytm10y = usDiffs >> take >> '_10y' >> takePanel
ytm30y = usDiffs >> take >> '_30y' >> takePanel
ytm2y5y =  usDiffs >> take >> ['_2y', '_5y']  >> takePanel
ytm5y10y =  usDiffs >> take >> ['_5y', '_10y']  >> takePanel
ytm10y30y = usDiffs >> take >> ['_10y', '_30y']  >> takePanel
ytm10y20y = usDiffs >> take >> ['_10y', '_20y']  >> takePanel
ytm2y5y10y = usDiffs >> take >> ['_2y', '_5y', '_10y']  >> takePanel
ytm3y5y10y = usDiffs >> take >> ['_3y', '_5y', '_10y']  >> takePanel     # maybe 3y is better than 2y?
ytm5y10y30y = usDiffs >> take >> ['_5y', '_10y', '_30y']  >> takePanel
ytm5y10y20y = usDiffs >> take >> ['_5y', '_10y', '_20y']  >> takePanel   # maybe 20y is better than 30y?
ytm2y5y10y30y = usDiffs >> take >> ['_2y', '_5y', '_10y', '_30y']  >> takePanel

In [8]:
"\n1) 7y ~ 2y" >> PP
showStats(ytm7y, ytm2y)

"\n2) 7y ~ 5y" >> PP
showStats(ytm7y, ytm5y)

"\n3) 7y ~ 10y" >> PP
showStats(ytm7y, ytm10y)

"\n4) 7y ~ 30y" >> PP
showStats(ytm7y, ytm30y)

"\n5) 7y ~ 2y,5y" >> PP
showStats(ytm7y, ytm2y5y)

"\n6) 7y ~ 5y,10y" >> PP
showStats(ytm7y, ytm5y10y)

"\n7) 7y ~ 10y,30y" >> PP
showStats(ytm7y, ytm10y30y)

"\n7b) 7y ~ 10y,20y" >> PP
showStats(ytm7y, ytm10y20y)

"\n8) 7y ~ 2y,5y,10y" >> PP
showStats(ytm7y, ytm2y5y10y)

"\n9) 7y ~ 3y,5y,10y" >> PP
showStats(ytm7y, ytm3y5y10y)

"\n10) 7y ~ 5y,10y,30y" >> PP
showStats(ytm7y, ytm5y10y30y)

"\n11) 7y ~ 5y,10y,20y" >> PP
showStats(ytm7y, ytm5y10y20y)

"\n12) 7y ~ 2y,5y,10y,30y" >> PP
showStats(ytm7y, ytm2y5y10y30y);

<br>

#### Analysis / conclusions

Reject models 5, 7b, 8, 9, 10, 11 & 12 because the p-stat is not significant.

![p-stats](xkcd-p-stats.png)

7 needs some further investigation (1, 2, 3, 4 and 6 are as anticipated).

In [9]:
"7) 7y ~ 10y,30y" >> PP
showStats(ytm7y, ytm10y30y);

In [10]:
"7) 7y ~ 10y,30y" >> PP
showStats(ytm7y, ytm10y30y, dict(addIntercept=True));

In [11]:
# sm.OLS(ytm7y, sm.add_constant(ytm10y30y)).fit().summary() >> PP;

In [12]:
"6) 7y ~ 5y,10y" >> PP
showStats(ytm7y, ytm5y10y);

In [13]:
"6) 7y ~ 5y,10y" >> PP
showStats(ytm7y, ytm5y10y, dict(addIntercept=True));dict(addIntercept=True)

<br>

In both cases clearly we shouldn't be adding an intercept.

We would select 6) in preference to 7) due to the F-test result and R-squared but also 7)'s betaHat is a long short hedge that implies a sort of mechanical relationship (like rates are pivotting like a see-saw on a 10y pivot) that is hard to believe in.

We should also investigate some constrained regressions, e.g. Least Angle Regression.

In [14]:
lm = showStats(ytm7y, ytm2y)
res = lm >> residuals(_, ytm7y, ytm2y)
res >> showResStats
res >> dm.plot.qq
res >> residualsPlot >> PP;

In [15]:
sm.OLS(ytm7y, sm.add_constant(ytm2y)).fit().summary() >> PP;

In [16]:
lm = showStats(ytm7y, ytm5y)
res = lm >> residuals(_, ytm7y, ytm5y)
res >> showResStats
res >> dm.plot.qq
res >> residualsPlot >> PP;

In [17]:
sm.OLS(ytm7y, sm.add_constant(ytm5y)).fit().summary() >> PP;

In [18]:
lm = showStats(ytm7y, ytm10y)
res = lm >> residuals(_, ytm7y, ytm10y)
res >> showResStats
res >> dm.plot.qq
res >> residualsPlot >> PP;

In [19]:
sm.OLS(ytm7y, sm.add_constant(ytm10y)).fit().summary() >> PP;

This one is a bit of a concern, we're getting some quantisation that we're not seeing in the regression against 5y. And the distribution is a bit squished.

In [20]:
lm = showStats(ytm7y, ytm30y)
res = lm >> residuals(_, ytm7y, ytm30y)
res >> showResStats
res >> dm.plot.qq
res >> residualsPlot >> PP;

In [21]:
sm.OLS(ytm7y, sm.add_constant(ytm30y)).fit().summary() >> PP;

In [22]:
lm = showStats(ytm7y, ytm5y10y)
res = lm >> residuals(_, ytm7y, ytm5y10y)
res >> showResStats
res >> dm.plot.qq
res >> residualsPlot >> PP;

In [23]:
sm.OLS(ytm7y, sm.add_constant(ytm5y10y)).fit().summary() >> PP;

<br>

### PCA

#### NB: Using SVD to perform PCA

References - https://www.quora.com/What-is-the-difference-between-PCA-and-SVD and https://en.wikipedia.org/wiki/Eigendecomposition_of_a_matrix

For a n x m panel matrix $P$, with n observations and m variables, let $C$ be the m x m covariance matrix $P ^ T P$. First let's consider the eigen decomposition of the covariance matrix - the resulting eigenvectors are the "principal components" and the eigen values the variance each eigen vector contributes to the overall variance.

A (nonzero) vector $v$ of dimension m is an eigenvector of a square m × m matrix $C$ if it satisfies a linear equation of the form:

$$
\begin{align}
C v = \lambda v
\end{align}
$$

Let $C$ be a square m × m matrix with m linearly independent eigenvectors $v_i$ (where i = 1, ..., m). Then $C$ can be factorized as


$$
C = V \Lambda  V^{-1}
$$


where $V$ is the square m × m matrix whose ith column is the eigenvector $v_i$ of $C$, and $\Lambda$ is the diagonal matrix whose diagonal elements are the corresponding eigenvalues, $\Lambda_ii = \lambda_i$.

$$
C V = V \Lambda
$$


Note that because  $P ^ T P$  is symmetric, $V$ is _**orthogonal**_ and the eigenvalues are all real-valued. This generates our "principal components" $V$ onto which we can project $P$ to get a set of uncorrelated variables:

$$P' = PV$$


Often we want $P'$ to be normalized, so dividing through by standard deviation, we get:

$$U = P' \Lambda ^{−1/2}$$ 

Substitute $PV$ for $P'$ and rearrange:

$$U = PV\Lambda^{-1/2}$$

$$P = U \Lambda^{1/2} V^T$$ 

i.e. the Singular Value Decomposition

<br>

#### PCA factors for the whole curve

In [24]:
panel = usDiffs >> drop >> ['_1m', '_2m', '_3m', '_6m'] >> takePanel
vor, s = dm.linalg.pca(panel)
s >> PP
xs = [1, 2, 3, 5, 7, 10, 20, 30]
plt.plot(xs, vor[:,0], 'red')
plt.plot(xs, vor[:,1], 'orange')
plt.plot(xs, vor[:,2], 'green')
plt.plot(xs, vor[:,3]);

<br>

#### PCA factors for the hedge points

In [25]:
panel = usDiffs >> take >> ['_2y', '_5y', '_10y', '_30y']  >> takePanel
vor, s = dm.linalg.pca(panel)
s >> PP
xs = [2, 5, 10, 30]
plt.plot(xs, vor[:,0], 'red')
plt.plot(xs, vor[:,1], 'orange')
plt.plot(xs, vor[:,2], 'green')
plt.plot(xs, vor[:,3]);

Factor 1 seems fairly consistent, factors 2 and 3 flip signs. showing that the set of selected inputs impacts the analysis (even with the orientation algo).